In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import numpy as np
import json
from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected
from google.colab import drive
import os

# ------------------------------------------------------------
# 0. Mount Google Drive
# ------------------------------------------------------------
try:
    drive.mount('/content/drive')
except:
    print("Drive already mounted or error mounting.")

# --- Define Your File Paths ---
DRIVE_PATH = "/content/drive/MyDrive/AFML_data/"
BERT_PATH = os.path.join(DRIVE_PATH, "bert")
GRAPH_OUTPUT_PATH = os.path.join(DRIVE_PATH, "graph")

# Create the output directory if it doesn't exist
os.makedirs(GRAPH_OUTPUT_PATH, exist_ok=True)

print(f"Loading data from: {DRIVE_PATH}")
print(f"Loading embeddings from: {BERT_PATH}")
print(f"Saving graph files to: {GRAPH_OUTPUT_PATH}")

# ------------------------------------------------------------
# 1. Load main dataset
# ------------------------------------------------------------
try:
    df = pd.read_csv(os.path.join(DRIVE_PATH, "TRAIN_model_ready_final_mode.csv"))
except FileNotFoundError:
    print("FATAL ERROR: 'TRAIN_model_ready_final_mode.csv' not found in your Drive.")
    print("Please run the final preprocessing script (Cell 6, v4) first.")
    exit()

print(f"Loaded dataset with {len(df)} reviews.")

# ------------------------------------------------------------
# 2. Build integer node mappings
# ------------------------------------------------------------
user_ids = df["user_id"].unique()
gmap_ids = df["gmap_id"].unique()

user_map = {uid: i for i, uid in enumerate(user_ids)}
gmap_map = {gid: i for i, gid in enumerate(gmap_ids)}

# Save maps to the NEW 'graph' folder
json.dump(user_map, open(os.path.join(GRAPH_OUTPUT_PATH, "user_map.json"), "w"))
json.dump(gmap_map, open(os.path.join(GRAPH_OUTPUT_PATH, "gmap_map.json"), "w"))
print(f"Saved 'user_map.json' and 'gmap_map.json' to {GRAPH_OUTPUT_PATH}.")

df["user_idx"] = df["user_id"].map(user_map)
df["gmap_idx"] = df["gmap_id"].map(gmap_map)

print(f"Users: {len(user_map)} | Businesses: {len(gmap_map)}")

# ------------------------------------------------------------
# 3. Create edge_index (user → business)
# ------------------------------------------------------------
edge_index = torch.tensor([
    df["user_idx"].values,
    df["gmap_idx"].values
], dtype=torch.long)

print(f"Created edge_index with shape {edge_index.shape}")

# ------------------------------------------------------------
# 4. Load Member 1 embeddings
# ------------------------------------------------------------
# NOTE: Make sure your file is named 'train_embeddings.npy' in the 'bert' folder
embeddings_path = os.path.join(BERT_PATH, "embeddings.npy")
try:
    embeddings = np.load(embeddings_path)
except FileNotFoundError:
    print(f"FATAL ERROR: '{embeddings_path}' not found.")
    print("Check your 'bert' folder. Is the file named 'embeddings.npy'?")
    exit()

assert embeddings.shape[0] == len(df), "Embedding size does not match df rows!"
print(f"Loaded embeddings: {embeddings.shape}")

# ------------------------------------------------------------
# 5. Build USER node features
# ------------------------------------------------------------
user_feature_cols = [
    "user_review_count",
    "user_mode_rating",
    "user_rating_variance",
    "user_gmap_diversity",
    "user_state_diversity"
]

user_feat = (
    df.groupby("user_idx")[user_feature_cols]
      .mean() # .mean() is correct here, it's getting the 1 row of stats per user
      .sort_index()
      .values
)

user_feat = torch.tensor(user_feat, dtype=torch.float)
print(f"user_feat shape: {user_feat.shape}")

# ------------------------------------------------------------
# 6. Build BUSINESS node features
# ------------------------------------------------------------
business_feature_cols = [
    "business_review_count",
    "business_mode_rating",
    "business_rating_variance"
]

biz_feat = (
    df.groupby("gmap_idx")[business_feature_cols]
      .mean() # .mean() is correct here, it's getting the 1 row of stats per business
      .sort_index()
      .values
)

biz_feat = torch.tensor(biz_feat, dtype=torch.float)
print(f"business_feat shape: {biz_feat.shape}")

# ------------------------------------------------------------
# 7. Build EDGE features (embeddings + review metadata)
# ------------------------------------------------------------
edge_feature_cols = [
    "rating",
    "review_hour",
    "review_age_days",
    "fast_travel" # This is your "fast_travel" boolean
]
print(f"Using edge features: {edge_feature_cols}")

edge_meta = df[edge_feature_cols].values
# Convert boolean 'is_diff_state' to float
edge_meta = edge_meta.astype(np.float32)

edge_attr = np.concatenate([embeddings, edge_meta], axis=1)

edge_attr = torch.tensor(edge_attr, dtype=torch.float)
print(f"edge_attr shape: {edge_attr.shape}")

# ------------------------------------------------------------
# 8. Build final HeteroData graph object
# ------------------------------------------------------------
data = HeteroData()

data["user"].x = user_feat
data["business"].x = biz_feat

data["user", "reviews", "business"].edge_index = edge_index
data["user", "reviews", "business"].edge_attr = edge_attr

print("Making graph undirected...")
data = ToUndirected()(data)
print("Graph is now undirected.")

# ------------------------------------------------------------
# 9. Save final graph
# ------------------------------------------------------------
FINAL_GRAPH_PATH = os.path.join(GRAPH_OUTPUT_PATH, "hetero_graph_train.pt")
torch.save(data, FINAL_GRAPH_PATH)

print(f"\n🎉 SUCCESS! Graph saved to your Google Drive as '{FINAL_GRAPH_PATH}'")
print("Give this file path to Member 4!")
print("\nFinal Graph Structure:")
print(data)
print("====================================================")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data from: /content/drive/MyDrive/AFML_data/
Loading embeddings from: /content/drive/MyDrive/AFML_data/bert
Saving graph files to: /content/drive/MyDrive/AFML_data/graph
Loaded dataset with 100008 reviews.
Saved 'user_map.json' and 'gmap_map.json' to /content/drive/MyDrive/AFML_data/graph.
Users: 51831 | Businesses: 3356
Created edge_index with shape torch.Size([2, 100008])
Loaded embeddings: (100008, 384)
user_feat shape: torch.Size([51831, 5])
business_feat shape: torch.Size([3356, 3])
Using edge features: ['rating', 'review_hour', 'review_age_days', 'fast_travel']
edge_attr shape: torch.Size([100008, 388])
Making graph undirected...
Graph is now undirected.

🎉 SUCCESS! Graph saved to your Google Drive as '/content/drive/MyDrive/AFML_data/graph/hetero_graph_train.pt'
Give this file path to Member 4!

Final Graph Structure:
HeteroData(
  user={ x=[51